# 1. Split the data into folds for training/validation/test

In [ ]:
import os
import sys

from functools import reduce

from pyment.data import NiftiDataset
from pyment.labels import BinaryLabel


data_folder = os.path.join(os.path.expanduser('~'), 'data', 'IXI', 'resized')
project_folder = os.path.join(os.path.expanduser('~'), 'projects', 'brain-sex')

if not os.path.isdir(project_folder):
    os.mkdir(project_folder)

if not os.path.isdir(os.path.join(project_folder, 'data')):
    os.mkdir(os.path.join(project_folder, 'data'))

    dataset = NiftiDataset.from_folder(data_folder, target='sex')
    label = BinaryLabel(name='sex', mapping={'M': 0, 'F': 1})
    label.fit(dataset.y)
    dataset.add_encoder('sex', label)
    
    test_folds = dataset.stratified_folds(5, variables=['sex', 'scanner', 'age'])
    test = test_folds[2]
    train = reduce(lambda x, y: x + y, [test_folds[i] for i in range(len(test_folds)) \
                                        if i != 2])

    folds = train.stratified_folds(5, variables=['sex', 'scanner', 'age'])
    print(folds)
    
    test.save(os.path.join(os.path.join(project_folder, 'data', 'test.json')))
    
    for i in range(len(folds)):
        folds[i].save(os.path.join(project_folder, 'data', f'fold_{i}.json'))

# 2. Configure a binary SFCN model

In [ ]:
import json

from tensorflow.keras.layers import Dense

from pyment.models import Model, RegressionSFCN

backbone = RegressionSFCN(input_shape=[43, 54, 41], weights='brain-age', include_top=False, 
                          dropout=0.5, weight_decay=1e-3)

head = Dense(1, activation='sigmoid')(backbone.output)

model = Model(backbone.input, head)

model = model.save(os.path.join(project_folder, 'model'))

# 3. Configure a preprocessor, an augmenter and a learning rate schedule

In [ ]:
from pyment.data.augmenters import NiftiAugmenter
from pyment.data.preprocessors import NiftiPreprocessor
from pyment.utils.learning_rate import LearningRateSchedule

# Create a preprocessor which normalizes the images to the range [0, 1]
preprocessor = NiftiPreprocessor(sigma=255.)
preprocessor.save(os.path.join(project_folder, 'preprocessor.json'))
print(preprocessor)

augmenter = NiftiAugmenter(flip_probabilities=[0.5, 0, 0])
augmenter.save(os.path.join(project_folder, 'augmenter.json'))
print(augmenter)

learning_rate_schedule = LearningRateSchedule({0: 1e-4, 20: 1e-5})
learning_rate_schedule.save(os.path.join(project_folder, 'learning_rate_schedule.json'))
print(learning_rate_schedule)

In [ ]:
from shutil import rmtree

from fit_model import fit_model

run_folder = os.path.join(project_folder, 'run')

if os.path.isdir(run_folder):
    rmtree(run_folder)
    
fit_model(model=os.path.join(project_folder, 'model'),
          training=[os.path.join(project_folder, 'data', f'fold_{i}.json') \
                    for i in range(4)],
          validation=[os.path.join(project_folder, 'data', f'fold_4.json')],
          preprocessor=os.path.join(project_folder, 'preprocessor.json'),
          augmenter=os.path.join(project_folder, 'augmenter.json'),
          batch_size=4,
          num_threads=8,
          loss='binary_crossentropy',
          metrics=['accuracy'],
          learning_rate_schedule=os.path.join(project_folder, 'learning_rate_schedule.json'),
          epochs=40,
          domain=None,
          destination=run_folder)